# Network Automation

Neste notebook estão presentes vários scripts que podem ser utilizados em automação de redes. 

Nota que estes scripts só funcionam para ExtremeXOS. Caso pretenda mudar algum script tenha em consideração que os comandos possam ser diferentes de fabricante para fabricante.

## Adicionar VLANs ao STP

In [ ]:
import exsh
import re

try:
    # Get the list of VLANs
    output = exsh.clicmd("show vlan", capture=True)
    vlans = re.findall(r"^(\d+)\s+", output, re.MULTILINE)

    for vlan in vlans:
        try:
            # Check if VLAN is already in STP
            stp_output = exsh.clicmd("show stp", capture=True)
            if f"VLAN {vlan}" not in stp_output:
                try:
                    # Add VLAN to STP
                    exsh.clicmd(f"configure stp vlan {vlan}")
                    print(f"[+] Successfully added VLAN {vlan} to STP")
                except Exception as e:
                    print(f"[-] Error adding VLAN {vlan} to STP: {e}")
            else:
                print(f"[*] VLAN {vlan} is already in STP")
        except Exception as e:
            print(f"[-] Error checking STP status for VLAN {vlan}: {e}")
except Exception as e:
    print(f"[-] Error retrieving VLANs: {e}")
    exit()


## Shut de Rogue DHCP Servers

In [ ]:
import exsh
import re

# Run DHCP snooping check
dhcp_output = exsh.clicmd("show ip-security dhcp-snooping violations", capture=True)

# Detect unauthorized DHCP servers
rogue_servers = []
for line in dhcp_output.split("\n"):
    match = re.search(r"(\S+)\s+(\S+)\s+Rogue", line)  # Extract IP and port
    if match:
        ip, port = match.groups()
        rogue_servers.append((ip, port))

# Disable rogue servers if found
if rogue_servers:
    for ip, port in rogue_servers:
        exsh.clicmd(f"disable port {port}")
        print(f"Rogue DHCP server detected on {port} (IP: {ip}) - Port Disabled!")
else:
    print("No rogue DHCP servers detected.")
